In [1]:
import numpy as np
import torch
import pyepo

# random seed
np.random.seed(42)
torch.manual_seed(42)

Auto-Sklearn cannot be imported.


### Number of Cores 

In [2]:
# number of nodes
nodes = [20, 50, 100, 200, 500]

###  Prediction Model

In [3]:
import torch
from torch import nn

# build linear model
class LinearRegression(nn.Module):

    def __init__(self, num_feat, num_node):
        super(LinearRegression, self).__init__()
        self.linear = nn.Linear(num_feat, num_node*(num_node-1)//2)

    def forward(self, x):
        out = self.linear(x)
        return out

### Training 

In [4]:
import copy
import time

import numpy as np
from tqdm import tqdm
from torch.utils.data import DataLoader
import pyepo
from pyepo.func import perturbedFenchelYoung

from model import tspDFJModel
from dataset import optDatasetConstrs
from func import innerConeAlignedCosine
from dataset import collate_fn

def pipeline():
    num_data = 10 # number of training data
    num_feat = 10 # size of feature
    deg = 4 # polynomial degree
    e = 0.5 # noise width
    batch_size = 32 # batch size
    elapseds_cave, elapseds_pfyl = [], []
    for num_node in nodes:
        print("Num of Node:", num_node)
        # set random seed
        np.random.seed(42)
        torch.manual_seed(42)
        # generate data
        feats, costs = pyepo.data.tsp.genData(num_data, num_feat, num_node, deg, e, seed=42)
        # set solver
        optmodel = tspDFJModel(num_node)
        # dataset and data loader
        dataset = optDatasetConstrs(optmodel, feats, costs=costs)
        dataloader = DataLoader(dataset, batch_size=batch_size, collate_fn=collate_fn, shuffle=True)
        # init model
        reg = LinearRegression(num_feat, num_node)
        # train
        print("CaVE+:")
        time.sleep(1)
        elapsed = trainingProcessCaVE(reg, optmodel, dataloader)
        elapseds_cave.append(elapsed / num_data)
        print()
        print("PFYL:")
        time.sleep(1)
        elapsed = trainingProcessPFYL(reg, optmodel, dataloader)
        elapseds_pfyl.append(elapsed / num_data)
        print()
        print()
    return elapseds_cave, elapseds_pfyl


def trainingProcessCaVE(reg, optmodel, dataloader):
    # copy predictor
    reg = copy.deepcopy(reg)
    # init loss
    ca_cos = innerConeAlignedCosine(optmodel, solver="clarabel")
    # train
    elapsed = trainCaVE(reg, ca_cos, dataloader)
    # print
    time.sleep(1)
    print("Test Elapsed Time: {:.2f} Sec".format(elapsed))
    return elapsed

        
def trainCaVE(reg, loss_func, dataloader):
    # set optimizer
    optimizer = torch.optim.Adam(reg.parameters(), lr=5e-2)
    # training
    tbar = tqdm(dataloader)
    tick = time.time()
    for data in tbar:
        # forward pass
        x, _, _, t_ctr = data
        # predict
        cp = reg(x)
        # loss
        loss = loss_func(cp, t_ctr)
        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        tbar.set_description("Loss: {:8.4f}".format(loss.item()))
    # record time
    tock = time.time()
    elapsed = tock - tick
    return elapsed

def trainingProcessPFYL(reg, optmodel, dataloader):
    # copy predictor
    reg = copy.deepcopy(reg)
    # init loss
    pfy = perturbedFenchelYoung(optmodel, n_samples=1, sigma=1.0)
    # train
    elapsed = trainPFYL(reg, pfy, dataloader)
    # print
    time.sleep(1)
    print("Test Elapsed Time: {:.2f} Sec".format(elapsed))
    return elapsed

        
def trainPFYL(reg, loss_func, dataloader):
    # set optimizer
    optimizer = torch.optim.Adam(reg.parameters(), lr=5e-2)
    # training
    tbar = tqdm(dataloader)
    tick = time.time()
    for data in tbar:
        # forward pass
        x, _, w, _ = data
        # predict
        cp = reg(x)
        # loss
        loss = loss_func(cp, w)
        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        tbar.set_description("Loss: {:8.4f}".format(loss.item()))
    # record time
    tock = time.time()
    elapsed = tock - tick
    return elapsed

(CVXPY) Dec 06 06:05:32 PM: Encountered unexpected exception importing solver OSQP:
ImportError('DLL load failed while importing qdldl: The specified module could not be found.')


### Plot

In [5]:
from matplotlib import pyplot as plt
import tol_colors as tc

def plot(nodes, elapseds_cave, elapsed_pfyl):
    # color map
    cset = tc.tol_cset('light')
    # time
    fig = plt.figure(figsize=(16,8))
    plt.plot(nodes, elapseds_cave, marker="o", linestyle="--", markersize=10, lw=5, color=cset.orange, label="CaVE+")
    plt.plot(nodes, elapsed_pfyl, marker="o", linestyle="--", markersize=10, lw=5, color=cset.light_blue, label="PFYL")
    plt.xlabel("Num of Nodes", fontsize=36)
    plt.ylabel("Runtime (Sec)", fontsize=36)
    plt.xticks(ticks=nodes, fontsize=28)
    plt.yticks(fontsize=28)
    plt.legend(fontsize=32)
    plt.title("Average Training Time per Instance", fontsize=36)
    plt.show()

### Run 

In [6]:
elapseds_cave, elapsed_pfyl = pipeline()

Num of Node: 20
Set parameter Username
Academic license - for non-commercial use only - expires 2024-01-01
Optimizing for optDataset...


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 63.51it/s]


CaVE+:
Num of cores: 1


  0%|                                                                                            | 0/1 [00:00<?, ?it/s]C:\Users\lucas\anaconda3\lib\site-packages\cvxpy\problems\problem.py:1403: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(
Loss:  -0.7984: 100%|████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.10s/it]


Test Elapsed Time: 1.10 Sec

PFYL:
Num of cores: 1


...

Test Elapsed Time: 3.61 Sec


Num of Node: 200
Optimizing for optDataset...


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [06:16<00:00, 37.68s/it]


CaVE+:
Num of cores: 1


  0%|                                                                                            | 0/1 [00:21<?, ?it/s]


RuntimeError: [enforce fail at C:\cb\pytorch_1000000000000\work\c10\core\impl\alloc_cpu.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 1616516800 bytes.

In [8]:
plot(nodes, elapseds_cave, elapsed_pfyl)

NameError: name 'elapseds_cave' is not defined